# BAYESIAN COMMODITY FUTURES INDICES

## Imports

In [ ]:
import pandas            as pd
import matplotlib.pyplot as plt
import seaborn           as sbn
import numpy             as np
import pyflux            as pf

from helpers                       import best_of
from itertools                     import product
from pandas.tools.plotting         import autocorrelation_plot
from scipy.stats                   import expon, laplace, t
from statsmodels.graphics.tsaplots import plot_pacf
from matplotlib.collections        import PolyCollection

## Notebook  Settings

In [ ]:
%matplotlib notebook
sbn.set_style('whitegrid')

## Read and Fix Data

In [ ]:
dat_file = './data/CommodityIndices.xlsx'
skiprows = (2, 3, 4, 5)
raw_data = pd.read_excel(dat_file, skiprows=skiprows, index_col=0, parse_cols='B:AE')

raw_data.columns = [column[4:-2].rstrip() for column in raw_data.columns]
raw_data.iloc[0] = 100.0

## Index, Returns, and Volatility
Pick any index by name

In [ ]:
raw_data.columns.values.tolist()

In [ ]:
name = 'Nickel'  # Name of index as string

data = raw_data[name].resample('BQ').mean()

fig, ax = plt.subplots(num=name+' Over the Aeons')
data.plot(ax=ax, label=name)

ax.legend(loc='best', frameon=True, fancybox=True, framealpha=0.8)
ax.set_xlabel('Time')
ax.set_ylabel('Relative index')

fig.tight_layout()

Not exactly what you'd call a _stationary_ time series. So, ...

### ... let's look at quarterly returns instead

In [ ]:
returns = 100 * data.pct_change(1).dropna()

fig = plt.figure(num='Returns on '+name+' and their (Partial) Autocorrelation', figsize=(9.7, 10))
return_ax = fig.add_subplot(311)
aucorr_ax = fig.add_subplot(312)
pacorr_ax = fig.add_subplot(313)

returns.plot(ax=return_ax)
return_ax.set_xlabel('Time')
return_ax.set_ylabel('Quarterly returns')

autocorrelation_plot(returns, ax=aucorr_ax)

plot_pacf(returns, alpha=0.01, ax=pacorr_ax, linewidth=0.5)
pacorr_ax.set_ylim(aucorr_ax.get_ylim())
pacorr_ax.set_xlim(aucorr_ax.get_xlim())
pacorr_ax.set_title('')
pacorr_ax.set_xlabel('Lag')
pacorr_ax.set_ylabel('Partial autocorrelation')

fig.tight_layout()

If neither autocorrelation nor partial autocorrelation significantly exceeds the confidence intervals at any lag, vanilla ARMA models are not going to do you much good. So let's also look at the *absolute* returns and _their_ (partial) autocorrelation next.

### Absolute returns and their autocorrelation

In [ ]:
fig = plt.figure(num='Absolute Returns on '+name+' and their (Partial) Autocorrelation', figsize=(9.7, 10))
return_ax = fig.add_subplot(311)
aucorr_ax = fig.add_subplot(312)
pacorr_ax = fig.add_subplot(313)

returns.abs().plot(ax=return_ax)
return_ax.set_xlabel('Time')
return_ax.set_ylabel('Absolute quarterly returns')

autocorrelation_plot(returns.abs(), ax=aucorr_ax);

plot_pacf(returns.abs(), alpha=0.05, ax=pacorr_ax, linewidth=0.5)
pacorr_ax.set_ylim(aucorr_ax.get_ylim())
pacorr_ax.set_xlim(aucorr_ax.get_xlim())
pacorr_ax.set_title('')
pacorr_ax.set_xlabel('Lag')
pacorr_ax.set_ylabel('Partial autocorrelation')

fig.tight_layout()

If either the autocorrelation or the partial autocorrelation significantly exceeds the confidence intervals at any lag, then GARCH-type models on the volatility might be what we are looking for.

### Distribution of (absolute) returns 

In [ ]:
fig = plt.figure(num='Distribution of (Absolute) Returns on '+name, figsize=(9.7, 4))
returns_ax = fig.add_subplot(121)
absrtns_ax = fig.add_subplot(122)

sbn.distplot(returns, 
             bins=15, 
             kde=False, 
             fit=laplace, 
             fit_kws={'color': '#c44e52', 'label': 'Laplace'}, 
             ax=returns_ax);
returns_ax.set_title('Quarterly Returns on '+name)
returns_ax.set_xlabel('Value')
returns_ax.set_ylabel('Frequency')

sbn.distplot(returns.abs(), 
             bins=15, 
             kde=False, 
             fit=expon, 
             fit_kws={'color': '#c44e52'}, 
             color='#55a868', 
             ax=absrtns_ax); 
absrtns_ax.set_title('Absolute Quarterly Returns on '+name)
absrtns_ax.set_xlabel('Absolute Value')
absrtns_ax.set_ylabel('Frequency')

x_lims = returns_ax.get_xlim()
x_vals = np.linspace(*x_lims, 200)
params = t.fit(returns)
returns_ax.plot(x_vals, t.pdf(x_vals, *params), label='Students-t')
returns_ax.legend(loc='best', frameon=True, fancybox=True, framealpha=0.8)

fig.tight_layout()

Note that the tails of the (absolute) returns are *far* too fat for a Gaussian process.

## A Grand Tour of the Bayesian  `PyFlux` Package
In case you are wondering why we never touch the _priors_, it is because they seem to be fine for the data at hand. Nothing could express my belief better. Note also, that we will not consider GARCH-type models here, as these are already dealt with in other notebooks at length.
### ARMA models
#### Set up lag ranges and scan through all models to find the one with minimal BIC

In [ ]:
maximumlag = 5  # Up to which lag-order should we test models?

timeseries = returns.values
drop_first = lambda lag_combi: timeseries[maximumlag - max(lag_combi.values()):]
lag_ranges = {'ar': range(maximumlag+1),
              'ma': range(maximumlag+1)}
lag_combis = (dict(zip(lag_ranges.keys(), values)) for values in product(*lag_ranges.values()))
ARMAmodels = (pf.ARIMA(drop_first(lags), **lags, family=pf.t()) for lags in lag_combis)

best_ARMA_model = best_of(ARMAmodels)
best_ARMA_model.fit().summary(transformed=False)

#### Perform Black Box Variational Inference and check its convergence
Consider increasing the number of iterations if you are not happy with what you see

In [ ]:
ARMAresults = best_ARMA_model.fit(method='BBVI', map_start=True, record_elbo=True, iterations=3000)

fig, ax = plt.subplots()
ax.plot(ARMAresults.elbo_records)
ax.set_xlabel('Iteration Number')
ax.set_ylabel('ELBO')

ARMAresults.summary(transformed=False)

#### Plot fit to returns and forecast for a year

In [ ]:
best_ARMA_model.plot_fit(figsize=(9.7, 5))
best_ARMA_model.plot_predict(h=4, figsize=(9.7, 5))
children = plt.gca().get_children() 
_ = [child.set_facecolor('C0') for child in children if isinstance(child, PolyCollection)]

### GARCH models
#### Set up model types and lag ranges to be scanned and find the one with minimal BIC
**Note**: SEGARCH and SEGARCHM models don't seem to always converge when fitted with MLE. Check before you include them!

In [ ]:
maximumlag = 4  # Up to which lag-order should we test models?

timeseries = returns.values
drop_first = lambda lag_combi: timeseries[maximumlag - max(lag_combi.values()):]
lag_ranges = {'p': range(1, maximumlag+1),
              'q': range(maximumlag+1)}
modeltypes = (pf.GARCH, pf.EGARCH, pf.LMEGARCH)  # and pf.SEGARCH, pf.SEGARCHM, pf.EGARCHM ?
lag_combis = (dict(zip(lag_ranges.keys(), values)) for values in product(*lag_ranges.values()))
ARCHmodels = [model(drop_first(lags), **lags) for lags in lag_combis for model in modeltypes]

best_model = best_of(ARCHmodels)
best_leveraged_model = best_of(ARCHmodels, leverage=True)

best_results = best_model.fit()
best_leveraged_results = best_leveraged_model.fit()

if best_results.bic < best_leveraged_results.bic:
    best_ARCH_model = best_model
    best_results.summary(transformed=False)
else:
    best_ARCH_model = best_leveraged_model
    best_leveraged_results.summary(transformed=False)

#### Perform Black Box Variational Inference and check its convergence
Consider increasing the number of iterations if you are not happy with what you see

In [ ]:
ARCHresults = best_ARCH_model.fit(method='M-H', map_start=True, record_elbo=True, nsims=30000)

#fig, ax = plt.subplots()
#ax.plot(ARCHresults.elbo_records)
#ax.set_xlabel('Iteration Number')
#ax.set_ylabel('ELBO')
best_ARCH_model.latent_variables.trace_plot(figsize=(9.7, 10))
ARCHresults.summary(transformed=False)

#### Plot fit to volatility and forecast for a year

In [ ]:
best_ARCH_model.plot_fit(figsize=(9.7, 5))
best_ARCH_model.plot_predict(h=4, figsize=(9.7, 5))
children = plt.gca().get_children() 
_ = [child.set_facecolor('C0') for child in children if isinstance(child, PolyCollection)]

### GAS models on the volatility ...
#### Set up lag ranges and scan through all models to find the one with minimal BIC

In [ ]:
maximumlag = 4  # Up to which lag-order should we test models?

timeseries = returns.abs().values
drop_first = lambda lag_combi: timeseries[maximumlag - max(lag_combi.values()):]
lag_ranges = {'ar': range(maximumlag+1),
              'sc': range(maximumlag+1)}
lag_combis = (dict(zip(lag_ranges.keys(), values)) for values in product(*lag_ranges.values()))
GAS_models = (pf.GAS(drop_first(lags), **lags, family=pf.Poisson()) for lags in lag_combis)

best_GAS_model = best_of(GAS_models)
best_GAS_model.fit().summary(transformed=False)

#### Sample posterior probability with MCMC
Consider increasing the number of simulated steps if the results are not converged

In [ ]:
GASresults = best_GAS_model.fit(method='M-H', map_start=True, record_elbo=True, nsims=20000)
#fig, ax = plt.subplots()
#ax.plot(GASresults.elbo_records)
#ax.set_xlabel('Iteration Number')
#ax.set_ylabel('ELBO')
best_GAS_model.latent_variables.trace_plot(figsize=(9.7, 10))
GASresults.summary(transformed=False)

#### Plot fit to volatility and forecast for a year

In [ ]:
best_GAS_model.plot_fit(figsize=(9.7, 8))
best_GAS_model.plot_predict(h=4, figsize=(9.7, 6))
children = plt.gca().get_children() 
_ = [child.set_facecolor('C0') for child in children if isinstance(child, PolyCollection)]

### ... and on the returns
#### Set up lag ranges and scan through all models to find the one with minimal BIC

In [ ]:
maximumlag = 4  # Up to which lag-order should we test models?

timeseries = returns.values
drop_first = lambda lag_combi: timeseries[maximumlag - max(lag_combi.values()):]
lag_ranges = {'ar': range(maximumlag+1),
              'sc': range(maximumlag+1)}
lag_combis = (dict(zip(lag_ranges.keys(), values)) for values in product(*lag_ranges.values()))
GAS_models = (pf.GAS(drop_first(lags), **lags, family=pf.t()) for lags in lag_combis)

best_GAS_model = best_of(GAS_models)
best_GAS_model.fit().summary(transformed=False)
#best_GAS_model.adjust_prior(4, pf.InverseGamma(10.0, 100.0))

#### Perform Black Box Variational Inference and check its convergence
Consider increasing the number of iterations if the you are not happy with what you see

In [ ]:
GASresults = best_GAS_model.fit(method='BBVI', map_start=True, record_elbo=True, iterations=3000)

fig, ax = plt.subplots()
ax.plot(GASresults.elbo_records)
ax.set_xlabel('Iteration Number')
ax.set_ylabel('ELBO')

GASresults.summary(transformed=False)

#### Plot fit to returns and forecast for a year

In [ ]:
best_GAS_model.plot_fit(figsize=(9.7, 8))
best_GAS_model.plot_predict(h=4, figsize=(9.7, 6))
children = plt.gca().get_children() 
_ = [child.set_facecolor('C0') for child in children if isinstance(child, PolyCollection)]

### GAS local-level model on the volatility
#### Find maximum-likelihood parameters first 

In [ ]:
timeseries = returns.abs().values
GASLLmodel = pf.GASLLEV(timeseries, pf.Poisson())
GASLLEVres = GASLLmodel.fit()
GASLLEVres.summary(transformed=False)

#### Sample posterior probability with MCMC
Consider increasing the number of simulated steps if the results are not converged

In [ ]:
GASLLEVres = GASLLmodel.fit(method='M-H', map_start=True, nsims=10000)
GASLLmodel.latent_variables.trace_plot(figsize=(9.7, 4))
GASLLEVres.summary(transformed=False)

#### Plot fit to volatility and forecast for a year

In [ ]:
GASLLmodel.plot_fit(figsize=(9.7, 8))
GASLLmodel.plot_predict(h=4, figsize=(9.7, 5))
children = plt.gca().get_children() 
_ = [child.set_facecolor('C0') for child in children if isinstance(child, PolyCollection)]

### GP-NARX models on the returns ...
#### Set up parameter ranges and scan through all models to find the one with minimal BIC
Note that not all kernels always lead to well-conditioned models

In [ ]:
max_lag = 4  # Up to which lag-order should we test models?

tseries = returns.values
without = lambda ar: tseries[max_lag - ar:]
ar_lags = range(1, max_lag+1)
kernels = (pf.SquaredExponential(), pf.OrnsteinUhlenbeck(), pf.Periodic())  # maybe also pf.RationalQuadratic()
GPNARXs = (pf.GPNARX(without(lag), lag, kernel) for lag in ar_lags for kernel in kernels)

best_GPNARX_model = best_of(GPNARXs)
nkernel = str(best_GPNARX_model.kernel).split('.')[3].split(' ')[0]
results = best_GPNARX_model.fit()
results.summary(transformed=False)
print('BEST KERNEL IS', nkernel)

#### Plot fit to returns and forecast for a year

In [ ]:
best_GPNARX_model.plot_fit(figsize=(9.7, 8))
best_GPNARX_model.plot_predict(h=4, figsize=(9.7, 5))
children = plt.gca().get_children() 
_ = [child.set_facecolor('C0') for child in children if isinstance(child, PolyCollection)]

### VAR model
#### Set up parameter ranges and scan through all models to find the one with minimal BIC

In [ ]:
maximumlag = 12  # Up to which lag-order should we test models?

timeseries = 100 * raw_data.loc[:, 'Energy':'Livestock'].resample('BQ').mean().pct_change(1).dropna()
drop_first = lambda ar_lag: timeseries.iloc[maximumlag - ar_lag:, :]
lag_orders = range(1, maximumlag+1)
VAR_models = (pf.VAR(drop_first(lag), lag) for lag in lag_orders)

min_bic, best_VAR_model = min((model.fit().bic, model) for model in VAR_models)
best_VAR_model.fit().summary(transformed=False)

#### Perform Black Box Variational Inference and check its convergence
Consider increasing the number of iterations if the you are not happy with what you see

In [ ]:
VARresults = best_VAR_model.fit(method='BBVI', map_start=True, record_elbo=True, iterations=5000)

fig, ax = plt.subplots()
ax.plot(VARresults.elbo_records)
ax.set_xlabel('Iteration Number')
ax.set_ylabel('ELBO')

VARresults.summary(transformed=False)

#### Plot fit to returns and forecast for a year

In [ ]:
best_VAR_model.plot_fit(figsize=(9.7, 4))
best_VAR_model.plot_predict(h=4, figsize=(9.7, 4))
for figure in map(plt.figure, plt.get_fignums()[-5:]):
    children = plt.gca().get_children()
    _ = [child.set_facecolor('C0') for child in children if isinstance(child, PolyCollection)]

## Concluding remarks

While there are yet more models available in the `PyFlux` package, it does not make sense to apply them all on the financial data at hand. In partiuclar, quarterly returns have neither a "local level" nor "local linear trend". Likewise, the raw index the data is not well described by such models because the volatility is not exactly constant. Lastly, the nonlinear local level models available in `PyFlux` do not describe the volatility well for some reason. So, while a fascinating class of models, they are not included here.

What could work: Brent Crude (no GAS on returns), Aluminum (no GAS on returns), Copper (no GAS on returns), Zinc(no Gas returns), Silver(no Gas returns), Wheat(no Gas returns)

Nickel is gut?